# 读取用户商品特征，生成可以训练的正负样本数据



In [2]:
import pickle
import numpy as np
import pandas as pd

In [3]:
TRAING_ADMIN_PATH = 'features/train_admin_features.pkl'
TRAING_ITEM_PATH = 'features/train_item_features.pkl'
TEST_ADMIN_PATH = 'features/test_admin_features.pkl'
TEST_ITEM_PATH = 'features/test_item_features.pkl'
TRAING_DATA_PATH = '../data/train.pkl'  # 读取原始训练集
TEST_DATA_PATH = '../data/test.pkl'

In [4]:
with open(TRAING_ADMIN_PATH,'rb') as f:
    train_admin_features = pickle.load(f)
with open(TRAING_ITEM_PATH,'rb') as f:
    train_item_features = pickle.load(f)
with open(TEST_ADMIN_PATH,'rb') as f:
    test_admin_features = pickle.load(f)
with open(TEST_ITEM_PATH,'rb') as f:
    test_item_features = pickle.load(f)

In [7]:
with open(TRAING_DATA_PATH,'rb') as f:
    train_data = pickle.load(f)

In [75]:
train_item_features[:4]

,item_id,num_sell,cate_id,store_id,item_price
0,1,1,2324,10013,4501
1,2,1,3882,4485,2751
2,3,2,155,8341,656
3,5,1,1191,1949,1689


In [9]:
train_data[:3]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend
0,xx,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,1,4,0
1,xx,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,1,4,0
2,yy,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,1,0,0


In [53]:
train_admin_features[:3]

,buyer_admin_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,1,28,27,0.000000,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0
1,29,42,34,0.261905,13924,4710,12,331.523810,146.5,22,32,14.0,20.0,8.0
2,71,10,8,0.300000,18547,6306,637,1854.700000,1175.0,7,7,0.0,2.0,8.0


In [68]:
def drop_lose_item(data_df,item_df, lose_item):
    length = len(list(lose_item))
    all_drop_index= []
    for i,item in enumerate(list(lose_item)):
        drop_index = data_df[data_df['item_id'] == item].index[:]
        all_drop_index.extend(list(drop_index.values))
    data_df_filter = data_df.drop(all_drop_index).reset_index(drop=True)  # 去掉item不在商品表中的数据
    data_df_filter = pd.merge(data_df_filter,item_df, how='left',on='item_id') # 将商品表中的数据添加到原始dataframe中   
    return data_df_filter

In [70]:
ITEMATTR_PKL_PATH = '../data/item_attr.pkl'
item_df = pickle.load(open(ITEMATTR_PKL_PATH, 'rb'), encoding='iso-8859-1')

In [71]:
# drop lose item
train_lose_item = set(train_data['item_id'].unique()).difference(set(item_df['item_id'].unique()))
train_filter = drop_lose_item(train_data, item_df, train_lose_item)

In [72]:
# merge admin and train_data
tmp = pd.merge(train_filter[['buyer_admin_id','item_id']],train_admin_features,how='outer',on=['buyer_admin_id'])

In [73]:
tmp.shape

(12843064, 15)

In [74]:
tmp[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,8362078,1,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0
1,8362078,3346056,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0
2,8362078,3530082,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0
3,8362078,4130245,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0


In [76]:
#merge admin and item
tmp2 = pd.merge(tmp, train_item_features,on=['item_id'],how='outer')

In [79]:
tmp2[tmp2['buyer_admin_id'] == 8362078]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,8362078,1,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,1,2324,10013,4501
1,8362078,3346056,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,28,1243,3185,168
29,8362078,3530082,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,2,559,38663,2051
31,8362078,4130245,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,41,1244,20576,125
72,8362078,4460989,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,5,2324,33106,8742
77,8362078,5841575,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,38,258,31024,390
115,8362078,5869190,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,47,3002,39455,97
116,8362078,5869190,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,47,3002,39455,97
162,8362078,6955075,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,9,2324,58195,127
163,8362078,6955075,12,10,0.0,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,9,2324,58195,127


In [80]:
TRAIN_PATH = 'model_data/train_data.pkl'
with open(TRAIN_PATH,'rb') as f:
    model_train_df = pickle.load(f)

In [81]:
model_train_df[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price,label
0,8362078,1,12,10,0.000000,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,1,2324,10013,4501,1
1,8362078,3346056,12,10,0.000000,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,28,1243,3185,168,1
2,2436524,3346056,21,21,0.238095,19744,9200,46,940.190476,497.0,9,17,0.0,20.0,1.0,28,1243,3185,168,1
3,2792675,3346056,25,25,0.120000,48236,19339,47,1929.440000,200.0,15,19,5.0,16.0,4.0,28,1243,3185,168,1


In [83]:
TEST_PATH = 'model_data/test_data.pkl'
with open(TEST_PATH,'rb') as f:
    model_test_df = pickle.load(f)

In [85]:
model_test_df[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price,label
0,1061132,189045,17,16,0.058824,6922,1033,27,407.176471,326.0,7,14,3.0,9.0,5,1,1506,11599,237,1
1,2129504,189045,7,3,0.000000,1755,269,237,250.714286,237.0,1,2,7.0,0.0,0,1,1506,11599,237,1
2,2129504,189045,7,3,0.000000,1755,269,237,250.714286,237.0,1,2,7.0,0.0,0,1,1506,11599,237,1
3,1061132,5655251,17,16,0.058824,6922,1033,27,407.176471,326.0,7,14,3.0,9.0,5,1,4016,65152,642,1
